# Introduction

Being a Certified Personal Trainer, I dreamt of owning my own Gym in future. One major success of running a Gym is selecting its location. You can have a world-class Gym but if it is in an inaccessible area, the Gym will not be visited and it will hardly take off. Thus, I would like to take this opportunity to find the most suitable location to set up a Gym

# Business Problem

Select a location which is accessible to the public and lack fitness facilities in the present day.

# Data

Since Singapore is well-connected by the Mass Rapid Transit (MRT) System, one key data is a pre-consolidated locations (Longtitude & Latitude) of all MRT Stations in Singapore. This data was consolidated and made available on GitHub, link provided below. An enquiry to Foursquare will locate the nearby fitness facilities and identify which MRT Station lack fitness facilities and will be a suitable Gym location. As the Venue Category in Foursquare might refer to fitness facilites as "Climbing Gym", "Yoga Studio", etc., it will be necessary to consider them as well before analyse and concluding on the location.

https://github.com/hxchua/datadoubleconfirm/blob/master/datasets/mrtsg.csv

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from botocore.client import Config
import ibm_boto3

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Methodology

## Step 1: Areas that are accessible by MRT are considered accessible, thus, the locations of all MRT Stations are imported and their headers are renamed. As the consolidated list also contained the Light Rail Transit (LRT) stations, a new dataframe (sg_mrt) was defined to only values with "MRT".

In [3]:
# save the DataFrame as CSV file
df_sg.to_csv("sg_df.csv", index=False)
df_sg.columns = ['ID','Neighborhood','Number','X','Y','Latitude', 'Longitude','Line']

In [4]:
df_sg

,ID,Neighborhood,Number,X,Y,Latitude,Longitude,Line
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED
3,81,BAKAU LRT STATION,SE3,36026.0821,41113.8766,1.388093,103.905418,OTHERS
4,80,BANGKIT LRT STATION,BP9,21248.2460,40220.9693,1.380018,103.772667,OTHERS
5,153,BARTLEY MRT STATION,CC12,33168.3039,36108.7003,1.342828,103.879746,YELLOW
6,115,BAYFRONT MRT STATION,DT16,30867.0093,29368.6250,1.281874,103.859073,BLUE
7,115,BAYFRONT MRT STATION,CE1,30867.0093,29368.6250,1.281874,103.859073,OTHERS
8,140,BEAUTY WORLD MRT STATION,DT5,21598.1665,35931.2359,1.341223,103.775810,BLUE
9,37,BEDOK MRT STATION,EW5,38757.9520,34024.7048,1.323980,103.929959,GREEN


In [5]:
MRT_names = list(df_sg.Neighborhood.unique())

sg_with_MRT = []

for x in MRT_names:
    if "MRT" in x.upper():
        sg_with_MRT.append(x)
        
sg_with_MRT

sg_mrt = df_sg[df_sg['Neighborhood'].isin(sg_with_MRT)].reset_index(drop=True)
sg_mrt.head()

,ID,Neighborhood,Number,X,Y,Latitude,Longitude,Line
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED
3,153,BARTLEY MRT STATION,CC12,33168.3039,36108.7003,1.342828,103.879746,YELLOW
4,115,BAYFRONT MRT STATION,DT16,30867.0093,29368.6250,1.281874,103.859073,BLUE


In [6]:
address = 'Singapore'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore are 1.357107, 103.8194992.


## Step 2: The Map is plotted based on the Latitide and Longitude above to ensure all MRT stations are represented in the data.

In [7]:
map_sg = folium.Map(location=[latitude, longitude],zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(sg_mrt['Latitude'], sg_mrt['Longitude'], sg_mrt['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sg) 

map_sg

In [8]:
# save the map as HTML file
map_sg.save('map_sg.html')

## Step 3: Using the Latitude and Longitude of the MRT Stations, data of the facilities nearby are retrieved from Foursquare.

In [10]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(sg_mrt['Latitude'], sg_mrt['Longitude'], sg_mrt['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [38]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(13594, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,ADMIRALTY MRT STATION,1.440585,103.800998,Kampung Admiralty Hawker Centre,1.439939,103.800774,Food Court
1,ADMIRALTY MRT STATION,1.440585,103.800998,Superpets Warehouse / HQ,1.436848,103.806059,Pet Store
2,ADMIRALTY MRT STATION,1.440585,103.800998,McDonald's,1.445931,103.798101,Fast Food Restaurant
3,ADMIRALTY MRT STATION,1.440585,103.800998,Asia Ghani,1.437431,103.795388,Malay Restaurant
4,ADMIRALTY MRT STATION,1.440585,103.800998,Starbucks,1.439761,103.800659,Coffee Shop


In [12]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
ADMIRALTY MRT STATION,81,81,81,81,81,81
ALJUNIED MRT STATION,100,100,100,100,100,100
ANG MO KIO MRT STATION,100,100,100,100,100,100
BARTLEY MRT STATION,100,100,100,100,100,100
BAYFRONT MRT STATION,200,200,200,200,200,200
BEAUTY WORLD MRT STATION,86,86,86,86,86,86
BEDOK MRT STATION,100,100,100,100,100,100
BEDOK NORTH MRT STATION,100,100,100,100,100,100
BEDOK RESERVOIR MRT STATION,100,100,100,100,100,100


In [13]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 329 uniques categories.


In [14]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Food Court', 'Pet Store', 'Fast Food Restaurant',
       'Malay Restaurant', 'Coffee Shop', 'Park', 'Sandwich Place',
       'College Theater', 'Supermarket', 'Food Truck',
       'Frozen Yogurt Shop', 'Clothing Store', 'Movie Theater',
       'Electronics Store', 'Grocery Store', 'Dim Sum Restaurant',
       'Sushi Restaurant', 'Café', 'Shopping Mall', 'Japanese Restaurant',
       'Trail', 'Chinese Restaurant', 'Indian Restaurant', 'Post Office',
       'Ice Cream Shop', 'Snack Place', 'Night Market',
       'Convenience Store', 'Gym', 'Thai Restaurant', 'Asian Restaurant',
       'Pizza Place', 'Bookstore', 'Mobile Phone Shop',
       'Seafood Restaurant', 'Restaurant', 'Bakery',
       'Fried Chicken Joint', 'BBQ Joint', 'Soup Place',
       'Cantonese Restaurant', 'Climbing Gym', 'Kitchen Supply Store',
       'Vegetarian / Vegan Restaurant', 'Gym / Fitness Center',
       'Bubble Tea Shop', 'Performing Arts Venue', 'Soccer Field',
       'Boarding House', 'Sporting Goods 

## Step 4: With the unique categories, a few categories like "Climbing Gym", "Yoga Studio", etc. are renamed to "Gym" to consider them as fitness facilities as well.

In [36]:
venues_df.replace('Gym / Fitness Center', 'Gym', inplace = True)
venues_df.replace('Climbing Gym', 'Gym', inplace = True)
venues_df.replace('Rock Climbing Spot', 'Gym', inplace = True)
venues_df.replace('Yoga Studio', 'Gym', inplace = True)
venues_df.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,ADMIRALTY MRT STATION,1.440585,103.800998,Kampung Admiralty Hawker Centre,1.439939,103.800774,Food Court
1,ADMIRALTY MRT STATION,1.440585,103.800998,Superpets Warehouse / HQ,1.436848,103.806059,Pet Store
2,ADMIRALTY MRT STATION,1.440585,103.800998,McDonald's,1.445931,103.798101,Fast Food Restaurant
3,ADMIRALTY MRT STATION,1.440585,103.800998,Asia Ghani,1.437431,103.795388,Malay Restaurant
4,ADMIRALTY MRT STATION,1.440585,103.800998,Starbucks,1.439761,103.800659,Coffee Shop


In [16]:
# check if the results contain "Gym"
"Gym" in venues_df['VenueCategory'].unique()

True

In [17]:
# one hot encoding
sg_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sg_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sg_onehot.columns[-1]] + list(sg_onehot.columns[:-1])
sg_onehot = sg_onehot[fixed_columns]

print(sg_onehot.shape)
sg_onehot.head()

(13594, 326)


,Neighborhoods,Accessories Store,Airport,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Casino,Cha Chaan Teng,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,College Stadium,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laundromat,Light Rail Station,Lighthouse,Lounge,Macanese Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Pakistani Restaurant,Park,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street Food Gatherin

In [34]:
sg_grouped = sg_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(sg_grouped.shape)
sg_grouped.head()

(121, 326)


,Neighborhoods,Accessories Store,Airport,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Court,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Casino,Cha Chaan Teng,Cheese Shop,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,College Stadium,College Theater,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Island,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laundromat,Light Rail Station,Lighthouse,Lounge,Macanese Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Pakistani Restaurant,Park,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Stationery Store,Steakhouse,Street Food Gatherin

In [19]:
len(sg_grouped[sg_grouped["Gym"] > 0])

103

## Step 5: Based on the data, 103 fitness facilities are identified. Now we look at where these facilities are by breaking them into 4 clusters.

In [20]:
sg_gym = sg_grouped[["Neighborhoods","Gym"]]
sg_gym.head()

,Neighborhoods,Gym
0,ADMIRALTY MRT STATION,0.012346
1,ALJUNIED MRT STATION,0.050000
2,ANG MO KIO MRT STATION,0.010000
3,BARTLEY MRT STATION,0.010000
4,BAYFRONT MRT STATION,0.020000


In [21]:
# set number of clusters
kclusters = 4

sg_clustering = sg_gym.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sg_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 1, 2, 2, 2, 3, 1, 1], dtype=int32)

In [22]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
sg_merged = sg_gym.copy()

# add clustering labels
sg_merged["Cluster Labels"] = kmeans.labels_

In [23]:
sg_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
sg_merged.head()

,Neighborhood,Gym,Cluster Labels
0,ADMIRALTY MRT STATION,0.012346,1
1,ALJUNIED MRT STATION,0.050000,0
2,ANG MO KIO MRT STATION,0.010000,1
3,BARTLEY MRT STATION,0.010000,1
4,BAYFRONT MRT STATION,0.020000,2


In [24]:
# merge sg_grouped with sg_mrt to add latitude/longitude for each neighborhood
sg_merged = sg_merged.join(sg_mrt.set_index("Neighborhood"), on="Neighborhood")

print(sg_merged.shape)
sg_merged.head() # check the last columns!

(144, 10)


,Neighborhood,Gym,Cluster Labels,ID,Number,X,Y,Latitude,Longitude,Line
0,ADMIRALTY MRT STATION,0.012346,1,12,NS10,24402.1063,46918.1131,1.440585,103.800998,RED
1,ALJUNIED MRT STATION,0.050000,0,16,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN
2,ANG MO KIO MRT STATION,0.010000,1,33,NS16,29807.2655,39105.7720,1.369933,103.849553,RED
3,BARTLEY MRT STATION,0.010000,1,153,CC12,33168.3039,36108.7003,1.342828,103.879746,YELLOW
4,BAYFRONT MRT STATION,0.020000,2,115,DT16,30867.0093,29368.6250,1.281874,103.859073,BLUE


In [35]:
# sort the results by Cluster Labels
print(sg_merged.shape)
sg_merged.sort_values(["Cluster Labels"], inplace=True)
sg_merged.head()

(144, 10)


,Neighborhood,Gym,Cluster Labels,ID,Number,X,Y,Latitude,Longitude,Line
69,MARINA SOUTH PIER MRT STATION,0.040000,0,150,NS28,31287.7063,28203.4939,1.271337,103.862853,RED
55,KALLANG MRT STATION,0.040000,0,26,EW10,32236.6038,32643.3302,1.311489,103.871377,GREEN
77,ONE-NORTH MRT STATION,0.040000,0,108,CC23,22896.0731,31346.4085,1.299760,103.787469,YELLOW
86,PUNGGOL MRT STATION,0.036585,0,106,NE17,35651.3846,42933.2694,1.404547,103.902052,PURPLE
106,TELOK AYER MRT STATION,0.040000,0,190,DT18,29667.6216,29414.5867,1.282290,103.848299,BLUE


# Results

The map below shows which of the MRT stations in 4 clusters. It shows which MRT stations are crowded with fitness facilities and which stations lack such facilities.

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sg_merged['Latitude'], sg_merged['Longitude'], sg_merged['Neighborhood'], sg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [27]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [28]:
sg_merged.loc[sg_merged['Cluster Labels'] == 0]

,Neighborhood,Gym,Cluster Labels,ID,Number,X,Y,Latitude,Longitude,Line
69,MARINA SOUTH PIER MRT STATION,0.040000,0,150,NS28,31287.7063,28203.4939,1.271337,103.862853,RED
36,DOVER MRT STATION,0.040000,0,29,EW22,21914.5509,32634.1186,1.311405,103.778652,GREEN
101,TAMPINES MRT STATION,0.040000,0,142,DT32,40213.0284,37463.3697,1.355077,103.943030,BLUE
79,OUTRAM PARK MRT STATION,0.050000,0,156,NE3,28696.2602,29135.3820,1.281405,103.839125,PURPLE
101,TAMPINES MRT STATION,0.040000,0,11,EW2,40445.4297,37266.9909,1.353301,103.945118,GREEN
100,TAMPINES EAST MRT STATION,0.050000,0,96,DT33,41501.0749,37586.6178,1.356191,103.954600,BLUE
79,OUTRAM PARK MRT STATION,0.050000,0,188,EW16,28646.3739,29316.7727,1.279765,103.839573,GREEN
97,STADIUM MRT STATION,0.060000,0,110,CC6,32677.7526,31687.0547,1.302841,103.875339,YELLOW
27,CHINATOWN MRT STATION,0.045000,0,131,NE4,29124.9585,29643.4786,1.284360,103.843424,PURPLE
27,CHINATOWN MRT STATION,0.045000,0,132,DT19,29238.9683,29686.5359,1.284749,103.844448,BLUE


In [29]:
sg_merged.loc[sg_merged['Cluster Labels'] == 1]

,Neighborhood,Gym,Cluster Labels,ID,Number,X,Y,Latitude,Longitude,Line
56,KEMBANGAN MRT STATION,0.010000,1,31,EW6,36862.1237,33699.3568,1.321038,103.912928,GREEN
38,ESPLANADE MRT STATION,0.010000,1,125,CC3,30469.0566,30634.4621,1.293322,103.855498,YELLOW
85,PROMENADE MRT STATION,0.010000,1,184,CC4,31080.6082,30623.0174,1.293218,103.860992,YELLOW
51,JALAN BESAR MRT STATION,0.010000,1,114,DT22,30466.0301,31970.4313,1.305404,103.855471,BLUE
85,PROMENADE MRT STATION,0.010000,1,102,DT15,31064.3414,30586.9146,1.292892,103.860846,BLUE
84,POTONG PASIR MRT STATION,0.010000,1,176,NE10,31977.1677,34842.7289,1.331380,103.869046,PURPLE
70,MARSILING MRT STATION,0.013158,1,36,NS8,21406.6874,46025.7686,1.432514,103.774090,RED
71,MARYMOUNT MRT STATION,0.010000,1,160,CC16,28679.3688,36758.7136,1.348707,103.839421,YELLOW
66,LORONG CHUAN MRT STATION,0.010000,1,162,CC14,31431.4054,37079.9444,1.351612,103.864143,YELLOW
65,LITTLE INDIA MRT STATION,0.010000,1,169,NE7,29817.1982,32124.8312,1.307198,103.848581,PURPLE


In [30]:
sg_merged.loc[sg_merged['Cluster Labels'] == 2]

,Neighborhood,Gym,Cluster Labels,ID,Number,X,Y,Latitude,Longitude,Line
74,NEWTON MRT STATION,0.020000,2,133,DT11,28499.9628,32877.5173,1.313607,103.837810,BLUE
118,YEW TEE MRT STATION,0.032258,2,6,NS5,18438.9791,42158.0124,1.397535,103.747431,RED
72,MATTAR MRT STATION,0.030000,2,138,DT25,33556.5834,34344.8585,1.326877,103.883234,BLUE
88,RAFFLES PLACE MRT STATION,0.020000,2,186,EW14,30019.1805,29617.6126,1.284126,103.851457,GREEN
108,TIONG BAHRU MRT STATION,0.020000,2,135,EW17,27298.9575,29846.2510,1.286193,103.827021,GREEN
113,UBI MRT STATION,0.020000,2,139,DT27,35337.7709,34685.4833,1.329957,103.899235,BLUE
109,TOA PAYOH MRT STATION,0.020000,2,174,NS19,29578.4408,34980.8638,1.332629,103.847498,RED
87,QUEENSTOWN MRT STATION,0.020000,2,21,EW19,24946.8083,30804.6785,1.294861,103.805891,GREEN
76,NOVENA MRT STATION,0.030000,2,147,NS20,29169.3298,33633.1535,1.320441,103.843823,RED
80,PASIR PANJANG MRT STATION,0.020000,2,155,CC26,23329.2647,28742.7658,1.276214,103.791361,YELLOW


In [31]:
sg_merged.loc[sg_merged['Cluster Labels'] == 3]

,Neighborhood,Gym,Cluster Labels,ID,Number,X,Y,Latitude,Longitude,Line
7,BEDOK NORTH MRT STATION,0.000,3,143,DT29,37421.7438,35214.6804,1.334742,103.917955,BLUE
61,KRANJI MRT STATION,0.000,3,4,NS7,20081.6974,45214.5479,1.425178,103.762187,RED
117,WOODLEIGH MRT STATION,0.000,3,146,NE11,32173.3186,35706.3794,1.339190,103.870808,PURPLE
112,TUAS WEST ROAD MRT STATION,0.000,3,27,EW32,6442.6401,34689.7988,1.329989,103.639666,GREEN
111,TUAS LINK MRT STATION,0.000,3,22,EW33,6138.3115,35848.0658,1.340464,103.636932,GREEN
110,TUAS CRESCENT MRT STATION,0.000,3,25,EW31,7495.5310,33698.6818,1.321027,103.649124,GREEN
14,BOTANIC GARDENS MRT STATION,0.000,3,159,DT9,26087.2663,33852.4484,1.322424,103.816136,BLUE
103,TAN KAH KEE MRT STATION,0.000,3,134,DT8,25106.8667,34234.9609,1.325883,103.807329,BLUE
52,JOO KOON MRT STATION,0.000,3,38,EW29,10756.3543,34438.2764,1.327717,103.678417,GREEN
104,TANAH MERAH MRT STATION,0.000,3,40,EW4,40578.8616,34379.4144,1.327187,103.946316,GREEN


# Discussion

Based on the analysis, Cluster 0 is the most crowded with fitness facilities while Cluster 3 lacks fitness facilities. This shows that Central Singapore and East Singapore are crowded with fitness facilities already and I should avoid those areas. While the western part of Singapore might have opportunity for me to set up a Gym.

# Conclsuion

With the help of Foursquare, I am able to get a quick sense of where the opportunities for a Gym are. It seems like Western Singapore, such as Jurong East, Chinese Gardens MRT Stations are possible areas for a new Gym.